<a href="https://colab.research.google.com/github/mbCap/Fallstudie_BESS_Degradation/blob/main/Latest_von_Fallstudie_BESS_Degradation_2_Zyklen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [ ]:
!git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
!pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")

T = range(1, 25)

# Parameter
cap = 40.0  # Nominelle Kapazität in [MWh]
DoD = 0.80  # Depth of Discharge
SOC_min = cap * (1 - DoD)  # Minimaler Ladezustand in [MWh]
SOC_max = cap  # Maximaler Ladezustand in [MWh]
c_rate = 0.5  # C-Rate
eta_rte = 0.975  # Round-trip Efficiency (einfache Richtung)
eta_wr = 0.985  # Wirkungsgrad Wechselrichter
cyclecost = 1500.0  # Fixkosten pro Zyklusdurchlauf
maxCycles = 2.0  # Maximal durchführbare Zyklen
p_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()  # Marktpreiserwartung zur Stunde t

# Lade- und Entladeverluste trennen
eta_charge = eta_rte * eta_wr  # Lade-Verlust
eta_discharge = eta_rte * eta_wr  # Entlade-Verlust

# Funktion zur Berechnung der degradierten Kapazität
def get_degraded_capacity(years):
    return cap * (1 - 0.02) ** years  # Annahme: 2% Degradation pro Jahr

# Funktion zur Initialisierung des Modells
def initialize_model(degraded_capacity):
    # Modell erstellen
    scip = Model()

    # Entscheidungsvariablen
    SOC = {t: scip.addVar(
        vtype="CONTINUOUS",
        lb=SOC_min,
        ub=SOC_max,
        name=f"SOC_{t}"
    ) for t in T}

    charge = {t: scip.addVar(
        vtype="CONTINUOUS",
        lb=0,
        ub=c_rate * degraded_capacity,
        name=f"charge_{t}"
    ) for t in T}

    discharge = {t: scip.addVar(
        vtype="CONTINUOUS",
        lb=0,
        ub=c_rate * degraded_capacity,
        name=f"discharge_{t}"
    ) for t in T}

    # Zählvariable für verbrauchte Zyklen
    Z = scip.addVar(
        vtype="CONTINUOUS",
        lb=0,
        ub=maxCycles,
        name="cycles_used"
    )

    # Nebenbedingungen
    scip.addCons(SOC[1] == 0.5 * degraded_capacity)
    scip.addCons(SOC[24] == 0.5 * degraded_capacity)
    scip.addCons(discharge[24] == 0)

    # Ladezustandsdynamik
    for t in T:
        if t == 1:
            continue
        scip.addCons(
            SOC[t]
            == SOC[t - 1]
            + eta_charge * charge[t - 1]
            - (1 / eta_discharge) * discharge[t - 1]
        )

    # Zyklenbegrenzung
    throughput = quicksum(charge[t] + discharge[t] for t in T)
    scip.addCons(
        Z >= throughput / (2.0 * degraded_capacity * DoD)
    )
    scip.addCons(
        Z <= maxCycles
    )

    # Zielfunktion
    scip.setObjective(quicksum(
        p_Markt[t - 1] * discharge[t] - p_Markt[t - 1] * charge[t]
        for t in T
    ) - cyclecost * Z, sense="maximize")

    return scip

# Berechnung des Erlöses pro Jahr und pro Tag
def calculate_revenue_for_year_and_day(years, scip):
    # Kapazität nach Degradation berechnen
    degraded_capacity = get_degraded_capacity(years)
    # Modell initialisieren mit degradiertem Kapazitätswert
    scip = initialize_model(degraded_capacity)

    # Modell lösen
    scip.optimize()

    # Erlös pro Tag
    daily_revenue = scip.getObjVal()
    # Erlös pro Jahr
    annual_revenue = daily_revenue * 365  # Multiplizieren mit 365 für das ganze Jahr

    return daily_revenue, annual_revenue

# Berechnung und Ausgabe des Erlöses für jedes Jahr und pro Tag
print(f"Erlös pro Jahr und pro Tag unter Berücksichtigung der Degradation:")

# Initialer Erlös für Jahr 0
revenue_today, annual_revenue_today = calculate_revenue_for_year_and_day(0, None)
print(f"Jahr 0: {revenue_today:.2f} € pro Tag / {annual_revenue_today:.2f} € pro Jahr")

# Berechnung und Ausgabe für die nächsten 10 Jahre
for years in range(1, 11):
    revenue_in_day, revenue_in_year = calculate_revenue_for_year_and_day(years, None)
    revenue_loss_per_day = revenue_today - revenue_in_day
    revenue_loss_per_year = annual_revenue_today - revenue_in_year
    print(f"Jahr {years}: {revenue_in_day:.2f} € pro Tag / {revenue_in_year:.2f} € pro Jahr "
          f"(Erlösverlust pro Tag: {revenue_loss_per_day:.2f} € / Erlösverlust pro Jahr: {revenue_loss_per_year:.2f} €)")

# Zusammenfassung der Veränderung des Erlöses pro Tag
revenue_loss_percentage = ((revenue_today - revenue_in_day) / revenue_today) * 100
print(f"\nWie verändert sich der Erlös eines Tages, wenn ich eine gewisse Degradation unterstelle (2% pro Jahr für die Batterie)?")
print(f"Der Erlös eines Tages sinkt aufgrund der Degradation im 10. Jahr um {revenue_loss_percentage:.2f}% im Vergleich zum initialen Jahr.")


Erlös pro Jahr und pro Tag unter Berücksichtigung der Degradation:
Jahr 0: 1885.12 € pro Tag / 688068.14 € pro Jahr
Jahr 1: 1873.13 € pro Tag / 683691.29 € pro Jahr (Erlösverlust pro Tag: 11.99 € / Erlösverlust pro Jahr: 4376.85 €)
Jahr 2: 1845.18 € pro Tag / 673490.04 € pro Jahr (Erlösverlust pro Tag: 39.94 € / Erlösverlust pro Jahr: 14578.10 €)
Jahr 3: 1816.36 € pro Tag / 662972.94 € pro Jahr (Erlösverlust pro Tag: 68.75 € / Erlösverlust pro Jahr: 25095.20 €)
Jahr 4: 1786.84 € pro Tag / 652195.68 € pro Jahr (Erlösverlust pro Tag: 98.28 € / Erlösverlust pro Jahr: 35872.46 €)
Jahr 5: 1756.59 € pro Tag / 641153.86 € pro Jahr (Erlösverlust pro Tag: 128.53 € / Erlösverlust pro Jahr: 46914.28 €)
Jahr 6: 1725.60 € pro Tag / 629842.99 € pro Jahr (Erlösverlust pro Tag: 159.52 € / Erlösverlust pro Jahr: 58225.16 €)
Jahr 7: 1693.86 € pro Tag / 618258.43 € pro Jahr (Erlösverlust pro Tag: 191.26 € / Erlösverlust pro Jahr: 69809.72 €)
Jahr 8: 1661.36 € pro Tag / 606395.46 € pro Jahr (Erlösverlust 